In [28]:

import numpy as np

In [ ]:
# Problems Notebook

In [ ]:
## Problem 1: Binary Words and Operations

### Introduction

SHA-256 (Secure Hash Algorithm 256-bit) is a cryptographic hash function that is part of the SHA-2 family, standardized by NIST in the [Federal Information Processing Standards Publication 180-4](link). The SHA-256 algorithm operates on 32-bit words (binary strings of length 32) and uses several bitwise logical functions and rotation operations to process input data.

In [ ]:
In this problem, we implement seven key functions defined on page 10 of the Secure Hash Standard:

**Logical Functions:**
- **Parity(x, y, z)** - Returns the bitwise XOR of three inputs
- **Ch(x, y, z)** - "Choose" function: uses x to choose bits from y or z
- **Maj(x, y, z)** - "Majority" function: returns the majority bit value at each position

**Rotation Functions:**
- **Σ₀²⁵⁶(x)** - Sigma0: combines three right rotations of x
- **Σ₁²⁵⁶(x)** - Sigma1: combines three right rotations of x  
- **σ₀²⁵⁶(x)** - sigma0: combines rotations and shifts for message schedule
- **σ₁²⁵⁶(x)** - sigma1: combines rotations and shifts for message schedule

All operations are performed on 32-bit unsigned integers, and we use NumPy's `uint32` data type to ensure proper handling of overflow and bitwise operations.

In [ ]:
### Implementation

In [ ]:
#### 1. Parity Function

The Parity function computes the bitwise XOR (exclusive OR) of three 32-bit words. According to the Secure Hash Standard (page 10), the Parity function is defined as:

**Parity(x, y, z) = x ⊕ y ⊕ z**

Where ⊕ represents the bitwise XOR operation. This function returns 1 for each bit position where an odd number of the inputs have a 1 bit, and 0 otherwise. The Parity function is used in SHA-1 for certain rounds of the compression function.

In [5]:
import numpy as np

def Parity(x, y, z):
    """
    Compute the bitwise Parity of three 32-bit words.
    
    The Parity function returns the bitwise XOR (exclusive OR) of x, y, and z.
    This is a logical function used in SHA-1 hash algorithm.
    
    Parameters
    ----------
    x : int or numpy.uint32
        First 32-bit word
    y : int or numpy.uint32
        Second 32-bit word
    z : int or numpy.uint32
        Third 32-bit word
    
    Returns
    -------
    numpy.uint32
        The bitwise XOR of x, y, and z
        
    Examples
    --------
    >>> Parity(0b1100, 0b1010, 0b1001)
    7
    >>> Parity(0xFFFFFFFF, 0xFFFFFFFF, 0xFFFFFFFF)
    4294967295
    """
    # Ensure all inputs are treated as 32-bit unsigned integers
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)
    
    # Compute bitwise XOR: x ⊕ y ⊕ z
    return x ^ y ^ z

In [ ]:
The implementation uses NumPy's `uint32` type to ensure all values are treated as 32-bit unsigned integers. The XOR operator `^` in Python performs bitwise XOR on each bit position independently.

In [ ]:
##### Testing Parity Function

In [7]:
# Test 1: binary example
# 1100 XOR 1010 = 0110, then 0110 XOR 1001 = 1111 (binary) = 15 (decimal)
result1 = Parity(0b1100, 0b1010, 0b1001)
print(f"Test 1: Parity(0b1100, 0b1010, 0b1001) = {result1}")
print(f"Expected: 15, Got: {result1}, Pass: {result1 == 15}")

Test 1: Parity(0b1100, 0b1010, 0b1001) = 15
Expected: 15, Got: 15, Pass: True


In [8]:
# Test 2: All bits set
# When all three inputs have all bits set, XOR returns all bits set
result2 = Parity(0xFFFFFFFF, 0xFFFFFFFF, 0xFFFFFFFF)
print(f"\nTest 2: Parity(0xFFFFFFFF, 0xFFFFFFFF, 0xFFFFFFFF) = {result2}")
print(f"Expected: {np.uint32(0xFFFFFFFF)}, Got: {result2}, Pass: {result2 == np.uint32(0xFFFFFFFF)}")


Test 2: Parity(0xFFFFFFFF, 0xFFFFFFFF, 0xFFFFFFFF) = 4294967295
Expected: 4294967295, Got: 4294967295, Pass: True


In [9]:
# Test 3: Zero inputs
# XOR of zeros is zero
result3 = Parity(0, 0, 0)
print(f"\nTest 3: Parity(0, 0, 0) = {result3}")
print(f"Expected: 0, Got: {result3}, Pass: {result3 == 0}")


Test 3: Parity(0, 0, 0) = 0
Expected: 0, Got: 0, Pass: True


In [10]:
# Test 4: Identity property - XOR with two identical values
# x XOR x XOR y = y (since x XOR x = 0)
x_val = np.uint32(0xABCDEF01)
y_val = np.uint32(0x12345678)
result4 = Parity(x_val, x_val, y_val)
print(f"\nTest 4: Parity(0xABCDEF01, 0xABCDEF01, 0x12345678) = {hex(result4)}")
print(f"Expected: {hex(y_val)}, Got: {hex(result4)}, Pass: {result4 == y_val}")


Test 4: Parity(0xABCDEF01, 0xABCDEF01, 0x12345678) = 0x12345678
Expected: 0x12345678, Got: 0x12345678, Pass: True


In [ ]:
#### 2. Ch (Choice) Function

The Ch function is a conditional function that "chooses" bits from y or z based on the corresponding bit in x. According to the Secure Hash Standard the Ch function is defined as:

**Ch(x, y, z) = (x ∧ y) ⊕ (¬x ∧ z)**

Where ∧ represents bitwise AND, ⊕ represents bitwise XOR, and ¬ represents bitwise NOT (complement).

The function works as follows: for each bit position, if the bit in x is 1, the result takes the bit from y; if the bit in x is 0, the result takes the bit from z. This is why it's called the "choice" function - x chooses between y and z.

In [15]:
def Ch(x, y, z):
    """
    Compute the Ch (Choice) function of three 32-bit words.
    
    The Ch function uses x as a selector to choose bits from either y or z.
    For each bit position: if x bit is 1, choose y bit; if x bit is 0, choose z bit.
    
    Formula: Ch(x, y, z) = (x ∧ y) ⊕ (¬x ∧ z)
    
    Parameters
    ----------
    x : int or numpy.uint32
        Selector word (32-bit)
    y : int or numpy.uint32
        First choice word (32-bit)
    z : int or numpy.uint32
        Second choice word (32-bit)
    
    Returns
    -------
    numpy.uint32
        Result of the choice function
        
    Examples
    --------
    >>> Ch(0b1111, 0b1010, 0b0101)
    10
    >>> Ch(0xFFFFFFFF, 0x12345678, 0xABCDEF01)
    305419896
    """
    # Ensure all inputs are treated as 32-bit unsigned integers
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)
    
    # Compute Ch(x, y, z) = (x ∧ y) ⊕ (¬x ∧ z)
    return (x & y) ^ (~x & z)

In [ ]:
The implementation follows the standard formula directly:
- `(x & y)`: selects bits from y where x has 1s
- `(~x & z)`: selects bits from z where x has 0s
- The XOR combines these two results

Since the two AND operations produce non-overlapping bit patterns (where x is 1 vs where x is 0), the XOR effectively acts as an OR, merging the selected bits.

In [ ]:
##### Testing Ch Function


In [16]:
# Test 1: Simple example where x chooses between y and z
# x = 1111 (all 1s) should select all bits from y = 1010
# Result should be 1010 (binary) = 10 (decimal)
result1 = Ch(0b1111, 0b1010, 0b0101)
print(f"Test 1: Ch(0b1111, 0b1010, 0b0101) = {result1}")
print(f"Expected: 10 (0b1010), Got: {result1}, Pass: {result1 == 10}")

Test 1: Ch(0b1111, 0b1010, 0b0101) = 10
Expected: 10 (0b1010), Got: 10, Pass: True


In [17]:
# Test 2: x = 0000 (all 0s) should select all bits from z = 0101
# Result should be 0101 (binary) = 5 (decimal)
result2 = Ch(0b0000, 0b1010, 0b0101)
print(f"\nTest 2: Ch(0b0000, 0b1010, 0b0101) = {result2}")
print(f"Expected: 5 (0b0101), Got: {result2}, Pass: {result2 == 5}")


Test 2: Ch(0b0000, 0b1010, 0b0101) = 5
Expected: 5 (0b0101), Got: 5, Pass: True


In [18]:
# Test 3: Mixed selection
# x = 1100, y = 1010, z = 0101
# Bits 0-1: x=00, select from z=01 -> 01
# Bits 2-3: x=11, select from y=10 -> 10
# Result: 1001 (binary) = 9 (decimal)
result3 = Ch(0b1100, 0b1010, 0b0101)
print(f"\nTest 3: Ch(0b1100, 0b1010, 0b0101) = {result3}")
print(f"Expected: 9 (0b1001), Got: {result3}, Pass: {result3 == 9}")


Test 3: Ch(0b1100, 0b1010, 0b0101) = 9
Expected: 9 (0b1001), Got: 9, Pass: True


In [19]:
# Test 4: Full 32-bit values
# When x = all 1s, result equals y
x_val = np.uint32(0xFFFFFFFF)
y_val = np.uint32(0x12345678)
z_val = np.uint32(0xABCDEF01)
result4 = Ch(x_val, y_val, z_val)
print(f"\nTest 4: Ch(0xFFFFFFFF, 0x12345678, 0xABCDEF01) = {hex(result4)}")
print(f"Expected: {hex(y_val)}, Got: {hex(result4)}, Pass: {result4 == y_val}")


Test 4: Ch(0xFFFFFFFF, 0x12345678, 0xABCDEF01) = 0x12345678
Expected: 0x12345678, Got: 0x12345678, Pass: True


In [20]:
# Test 5: When x = all 0s, result equals z
x_val = np.uint32(0x00000000)
y_val = np.uint32(0x12345678)
z_val = np.uint32(0xABCDEF01)
result5 = Ch(x_val, y_val, z_val)
print(f"\nTest 5: Ch(0x00000000, 0x12345678, 0xABCDEF01) = {hex(result5)}")
print(f"Expected: {hex(z_val)}, Got: {hex(result5)}, Pass: {result5 == z_val}")


Test 5: Ch(0x00000000, 0x12345678, 0xABCDEF01) = 0xabcdef01
Expected: 0xabcdef01, Got: 0xabcdef01, Pass: True


In [ ]:
#### 3. Maj (Majority) Function

The Maj function returns the majority value for each bit position across three 32-bit words. According to the Secure Hash Standard (page 10), the Maj function is defined as:

**Maj(x, y, z) = (x ∧ y) ⊕ (x ∧ z) ⊕ (y ∧ z)**

Where ∧ represents bitwise AND and ⊕ represents bitwise XOR.

For each bit position, the function returns 1 if at least two of the three input bits are 1, and returns 0 otherwise. This implements a majority vote at each bit position independently.

In [21]:
def Maj(x, y, z):
    """
    Compute the Maj (Majority) function of three 32-bit words.
    
    The Maj function returns the majority bit value at each bit position.
    If at least two of the three bits are 1, the result is 1; otherwise 0.
    
    Formula: Maj(x, y, z) = (x ∧ y) ⊕ (x ∧ z) ⊕ (y ∧ z)
    
    Parameters
    ----------
    x : int or numpy.uint32
        First 32-bit word
    y : int or numpy.uint32
        Second 32-bit word
    z : int or numpy.uint32
        Third 32-bit word
    
    Returns
    -------
    numpy.uint32
        Result where each bit is the majority of the corresponding input bits
        
    Examples
    --------
    >>> Maj(0b1110, 0b1100, 0b1000)
    12
    >>> Maj(0xFFFFFFFF, 0xFFFFFFFF, 0x00000000)
    4294967295
    """
    # Ensure all inputs are treated as 32-bit unsigned integers
    x = np.uint32(x)
    y = np.uint32(y)
    z = np.uint32(z)
    
    # Compute Maj(x, y, z) = (x ∧ y) ⊕ (x ∧ z) ⊕ (y ∧ z)
    return (x & y) ^ (x & z) ^ (y & z)

In [ ]:
The implementation follows the standard formula. The three AND operations identify positions where pairs of inputs both have 1s, and the XOR operations combine these results. This formula correctly implements the majority logic:
- If all three bits are 1: `(1∧1) ⊕ (1∧1) ⊕ (1∧1) = 1 ⊕ 1 ⊕ 1 = 1`
- If two bits are 1: `(1∧1) ⊕ (1∧0) ⊕ (1∧0) = 1 ⊕ 0 ⊕ 0 = 1`
- If one bit is 1: `(0∧1) ⊕ (0∧0) ⊕ (1∧0) = 0 ⊕ 0 ⊕ 0 = 0`
- If no bits are 1: `(0∧0) ⊕ (0∧0) ⊕ (0∧0) = 0 ⊕ 0 ⊕ 0 = 0`

In [ ]:
##### Testing Maj Function

In [22]:
# Test 1: All three bits are 1 at each position
# Result should be all 1s
result1 = Maj(0b1111, 0b1111, 0b1111)
print(f"Test 1: Maj(0b1111, 0b1111, 0b1111) = {result1}")
print(f"Expected: 15 (0b1111), Got: {result1}, Pass: {result1 == 15}")

Test 1: Maj(0b1111, 0b1111, 0b1111) = 15
Expected: 15 (0b1111), Got: 15, Pass: True


In [23]:
# Test 2: Two inputs have all 1s, one has all 0s
# Majority is 1 at each position, so result is all 1s
result2 = Maj(0b1111, 0b1111, 0b0000)
print(f"\nTest 2: Maj(0b1111, 0b1111, 0b0000) = {result2}")
print(f"Expected: 15 (0b1111), Got: {result2}, Pass: {result2 == 15}")


Test 2: Maj(0b1111, 0b1111, 0b0000) = 15
Expected: 15 (0b1111), Got: 15, Pass: True


In [24]:
# Test 3: Mixed bits - majority voting
# x = 1110, y = 1100, z = 1000
# Bit 0: (0,0,0) -> majority 0
# Bit 1: (1,0,0) -> majority 0
# Bit 2: (1,1,0) -> majority 1
# Bit 3: (1,1,1) -> majority 1
# Result: 1100 (binary) = 12 (decimal)
result3 = Maj(0b1110, 0b1100, 0b1000)
print(f"\nTest 3: Maj(0b1110, 0b1100, 0b1000) = {result3}")
print(f"Expected: 12 (0b1100), Got: {result3}, Pass: {result3 == 12}")


Test 3: Maj(0b1110, 0b1100, 0b1000) = 12
Expected: 12 (0b1100), Got: 12, Pass: True


In [25]:
# Test 4: All zeros
# Majority is 0 at each position
result4 = Maj(0b0000, 0b0000, 0b0000)
print(f"\nTest 4: Maj(0b0000, 0b0000, 0b0000) = {result4}")
print(f"Expected: 0, Got: {result4}, Pass: {result4 == 0}")


Test 4: Maj(0b0000, 0b0000, 0b0000) = 0
Expected: 0, Got: 0, Pass: True


In [26]:
# Test 5: Full 32-bit test - two identical values
# When two inputs are the same, result equals that value (majority rule)
x_val = np.uint32(0xABCDEF01)
y_val = np.uint32(0xABCDEF01)
z_val = np.uint32(0x12345678)
result5 = Maj(x_val, y_val, z_val)
print(f"\nTest 5: Maj(0xABCDEF01, 0xABCDEF01, 0x12345678) = {hex(result5)}")
print(f"Expected: {hex(x_val)}, Got: {hex(result5)}, Pass: {result5 == x_val}")


Test 5: Maj(0xABCDEF01, 0xABCDEF01, 0x12345678) = 0xabcdef01
Expected: 0xabcdef01, Got: 0xabcdef01, Pass: True


In [27]:
# Test 6: Verify bitwise majority with specific pattern
# x = 1010, y = 1100, z = 0110
# Bit 0: (0,0,0) -> 0
# Bit 1: (1,0,1) -> 1
# Bit 2: (0,1,1) -> 1
# Bit 3: (1,1,0) -> 1
# Result: 1110 (binary) = 14 (decimal)
result6 = Maj(0b1010, 0b1100, 0b0110)
print(f"\nTest 6: Maj(0b1010, 0b1100, 0b0110) = {result6}")
print(f"Expected: 14 (0b1110), Got: {result6}, Pass: {result6 == 14}")


Test 6: Maj(0b1010, 0b1100, 0b0110) = 14
Expected: 14 (0b1110), Got: 14, Pass: True


In [20]:
#### 4. Σ₀²⁵⁶ (Sigma0) Function

The Σ₀²⁵⁶ function is one of the rotation functions used in SHA-256. According to the Secure Hash Standard (page 10), Sigma0 is defined as:

**Σ₀²⁵⁶(x) = ROTR²(x) ⊕ ROTR¹³(x) ⊕ ROTR²²(x)**

Where ROTR^n(x) represents a circular right rotation of x by n bit positions, and ⊕ represents bitwise XOR.

A circular right rotation moves bits to the right, with bits that fall off the right end wrapping around to the left end. This function combines three different rotations of the input to create a non-linear transformation used in the SHA-256 compression function.

SyntaxError: invalid character '₀' (U+2080) (1931488867.py, line 3)

In [21]:
##implementing a helper function for right rotation:

In [30]:
def ROTR(x, n, word_size=32):
    """
    Perform a circular right rotation on a word.
    
    Rotates the bits of x to the right by n positions. Bits that fall off
    the right end wrap around to the left end.
    
    Parameters
    ----------
    x : int or numpy.uint32
        The word to rotate
    n : int
        Number of positions to rotate right
    word_size : int, optional
        Size of the word in bits (default is 32)
    
    Returns
    -------
    numpy.uint32
        The rotated word
        
    Examples
    --------
    >>> ROTR(0b11010000, 2, word_size=8)
    52
    """
    x = np.uint32(x)
    n = n % word_size  # Handle n >= word_size
    
    # Create a mask for the word size
    mask = np.uint32((1 << word_size) - 1)
    
    # Right rotation: (x >> n) | (x << (word_size - n))
    # Apply mask to ensure we only keep bits within word_size
    result = ((x >> n) | (x << (word_size - n))) & mask
    
    return np.uint32(result)

In [23]:
The rotation works by:
1. Shifting bits right by n positions: `x >> n`
2. Shifting bits left by `(word_size - n)` positions: `x << (word_size - n)` to wrap the bits
3. Combining with OR: the right shift puts bits in their new positions, and the left shift wraps the overflow bits

For example, rotating `11010000` right by 2 positions:
- Right shift by 2: `00110100` (top 2 bits lost)
- Left shift by 30: `00000000` (bottom 30 bits lost, top 2 bits = `00`)
- OR them together: `00110100` = 52

SyntaxError: leading zeros in decimal integer literals are not permitted; use an 0o prefix for octal integers (4141160003.py, line 7)

In [24]:
##implementing Sigma0 using the rotation function:

In [25]:
def Sigma0(x):
    """
    Compute the Σ₀²⁵⁶ function for SHA-256.
    
    Sigma0 combines three right rotations of the input word using XOR.
    This function is used in the SHA-256 compression function.
    
    Formula: Σ₀²⁵⁶(x) = ROTR²(x) ⊕ ROTR¹³(x) ⊕ ROTR²²(x)
    
    Parameters
    ----------
    x : int or numpy.uint32
        Input 32-bit word
    
    Returns
    -------
    numpy.uint32
        Result of the Sigma0 transformation
        
    Examples
    --------
    >>> Sigma0(0x12345678)
    1293428941
    """
    x = np.uint32(x)
    
    # Compute ROTR²(x) ⊕ ROTR¹³(x) ⊕ ROTR²²(x)
    return ROTR(x, 2) ^ ROTR(x, 13) ^ ROTR(x, 22)

In [26]:
##### Testing Sigma0 Function

In [31]:
# Test ROTR helper function
# Rotate 11010000 (208) right by 2 positions
# Expected: 00110100 (52)
test_rotr = ROTR(0b11010000, 2, word_size=8)  # Using 8-bit for clarity
print(f"ROTR Test: ROTR(0b11010000, 2) = {test_rotr}")
print(f"Binary: {bin(test_rotr)}")
print(f"Expected: 52 (0b110100), Got: {test_rotr}, Pass: {test_rotr == 52}")

ROTR Test: ROTR(0b11010000, 2) = 52
Binary: 0b110100
Expected: 52 (0b110100), Got: 52, Pass: True


In [32]:
# Test 1: Sigma0 with a known value
# We'll verify that the function executes without error
x_val = np.uint32(0x12345678)
result1 = Sigma0(x_val)
print(f"\nTest 1: Sigma0(0x12345678) = {result1} (0x{result1:08x})")
print(f"Type check: {type(result1)} - Pass: {isinstance(result1, np.uint32)}")


Test 1: Sigma0(0x12345678) = 1712612468 (0x66146474)
Type check: <class 'numpy.uint32'> - Pass: True


In [33]:
# Test 2: Sigma0 with all zeros
# Rotation of zero is zero, XOR of zeros is zero
result2 = Sigma0(0x00000000)
print(f"\nTest 2: Sigma0(0x00000000) = {result2}")
print(f"Expected: 0, Got: {result2}, Pass: {result2 == 0}")


Test 2: Sigma0(0x00000000) = 0
Expected: 0, Got: 0, Pass: True


In [34]:
# Test 3: Sigma0 with all ones
# Should produce a specific pattern due to rotations
result3 = Sigma0(0xFFFFFFFF)
print(f"\nTest 3: Sigma0(0xFFFFFFFF) = {result3} (0x{result3:08x})")
# All rotations of all 1s gives all 1s, XOR of identical values gives 0
# ROTR(0xFFFFFFFF, n) = 0xFFFFFFFF for any n
# So: 0xFFFFFFFF ^ 0xFFFFFFFF ^ 0xFFFFFFFF = 0xFFFFFFFF
print(f"Expected: {np.uint32(0xFFFFFFFF)}, Got: {result3}, Pass: {result3 == np.uint32(0xFFFFFFFF)}")


Test 3: Sigma0(0xFFFFFFFF) = 4294967295 (0xffffffff)
Expected: 4294967295, Got: 4294967295, Pass: True


In [35]:
# Test 4: Verify rotation amounts by testing with single bit
# x = 0x00000001 (bit 0 set)
# ROTR(x, 2) = 0x40000000 (bit 30 set)
# ROTR(x, 13) = 0x00080000 (bit 19 set)  
# ROTR(x, 22) = 0x00000400 (bit 10 set)
x_val = np.uint32(0x00000001)
result4 = Sigma0(x_val)
expected4 = np.uint32(0x40000000) ^ np.uint32(0x00080000) ^ np.uint32(0x00000400)
print(f"\nTest 4: Sigma0(0x00000001) = 0x{result4:08x}")
print(f"Expected: 0x{expected4:08x}, Got: 0x{result4:08x}, Pass: {result4 == expected4}")


Test 4: Sigma0(0x00000001) = 0x40080400
Expected: 0x40080400, Got: 0x40080400, Pass: True


In [ ]:
#### 5. Σ₁²⁵⁶ (Sigma1) Function

The Σ₁²⁵⁶ function is another rotation function used in SHA-256. According to the Secure Hash Standard (page 10), Sigma1 is defined as:

**Σ₁²⁵⁶(x) = ROTR⁶(x) ⊕ ROTR¹¹(x) ⊕ ROTR²⁵(x)**

Where ROTR^n(x) represents a circular right rotation of x by n bit positions, and ⊕ represents bitwise XOR.

Like Sigma0, this function combines three different rotations, but uses different rotation amounts (6, 11, and 25). This function is also used in the SHA-256 compression function to provide non-linear transformations.

In [36]:
def Sigma1(x):
    """
    Compute the Σ₁²⁵⁶ function for SHA-256.
    
    Sigma1 combines three right rotations of the input word using XOR.
    This function is used in the SHA-256 compression function.
    
    Formula: Σ₁²⁵⁶(x) = ROTR⁶(x) ⊕ ROTR¹¹(x) ⊕ ROTR²⁵(x)
    
    Parameters
    ----------
    x : int or numpy.uint32
        Input 32-bit word
    
    Returns
    -------
    numpy.uint32
        Result of the Sigma1 transformation
        
    Examples
    --------
    >>> Sigma1(0x12345678)
    1998951682
    """
    x = np.uint32(x)
    
    # Compute ROTR⁶(x) ⊕ ROTR¹¹(x) ⊕ ROTR²⁵(x)
    return ROTR(x, 6) ^ ROTR(x, 11) ^ ROTR(x, 25)

In [ ]:
##### Testing Sigma1 Function

In [ ]:
# Test 1: Sigma1 with a known value
# We'll verify that the function executes without error
x_val = np.uint32(0x12345678)
result1 = Sigma1(x_val)
print(f"Test 1: Sigma1(0x12345678) = {result1} (0x{result1:08x})")
print(f"Type check: {type(result1)} - Pass: {isinstance(result1, np.uint32)}")

In [ ]:
# Test 2: Sigma1 with all zeros
# Rotation of zero is zero, XOR of zeros is zero
result2 = Sigma1(0x00000000)
print(f"\nTest 2: Sigma1(0x00000000) = {result2}")
print(f"Expected: 0, Got: {result2}, Pass: {result2 == 0}")

In [ ]:
# Test 3: Sigma1 with all ones
# All rotations of all 1s gives all 1s
# 0xFFFFFFFF ^ 0xFFFFFFFF ^ 0xFFFFFFFF = 0xFFFFFFFF
result3 = Sigma1(0xFFFFFFFF)
print(f"\nTest 3: Sigma1(0xFFFFFFFF) = {result3} (0x{result3:08x})")
print(f"Expected: {np.uint32(0xFFFFFFFF)}, Got: {result3}, Pass: {result3 == np.uint32(0xFFFFFFFF)}")

In [ ]:
# Test 4: Verify rotation amounts by testing with single bit
# x = 0x00000001 (bit 0 set)
# ROTR(x, 6) = 0x04000000 (bit 26 set)
# ROTR(x, 11) = 0x00100000 (bit 21 set)  
# ROTR(x, 25) = 0x00000080 (bit 7 set)
x_val = np.uint32(0x00000001)
result4 = Sigma1(x_val)
expected4 = np.uint32(0x04000000) ^ np.uint32(0x00100000) ^ np.uint32(0x00000080)
print(f"\nTest 4: Sigma1(0x00000001) = 0x{result4:08x}")
print(f"Expected: 0x{expected4:08x}, Got: 0x{result4:08x}, Pass: {result4 == expected4}")

In [ ]:
# Test 5: Different value to ensure distinct behavior from Sigma0
x_val = np.uint32(0xABCDEF01)
result5 = Sigma1(x_val)
print(f"\nTest 5: Sigma1(0xABCDEF01) = 0x{result5:08x}")
# Just verify it returns a uint32 and is different from the input
print(f"Type check: Pass: {isinstance(result5, np.uint32)}")
print(f"Result differs from input: Pass: {result5 != x_val}")